In [1]:
from const import *

### 1. Cloud priority

In [2]:
total_ru_services = [
    [4, 1, 0],
    [3, 1, 1],
    [2, 1, 2],
    [0, 1, 4],
    [1, 1, 3],
    [0, 0, 5],
    [2, 2, 1],
    [3, 2, 0],
]

cloud_capacity = 40_000
fog_capacity = [20_000, 20_000]
cloud_energy = 600
fog_energy = 300

band_division = [{CLOUD: 1966, FOG: 0}, {CLOUD: 674.4, FOG: 1291.6}, {CLOUD: 74, FOG: 1892}]

In [3]:
lost_packeges = 0
cloud_used_capacity = 0
fog_used_capacity = [0, 0]
total_energy_consumption = 0
active = {"cloud": False, "fog_1": False, "fog_2": False}

cloud_band = []
fog_band = []
lost_packeges_all = []
energy_consumption = []

for services in total_ru_services:
    embb_num = services[EMBB]
    mmtc_num = services[MMTC]
    urllc_num = services[URLLC]

    # Process eMBB services
    # Allocate as much as possible using split E
    rest_cloud = cloud_capacity - cloud_used_capacity
    can_allocate = min(rest_cloud // band_division[E][CLOUD], embb_num)
    cloud_used_capacity += can_allocate * band_division[E][CLOUD]
    embb_num -= can_allocate

    if embb_num != 0:
        # Allocate as much as possible using split I
        rest_cloud = cloud_capacity - cloud_used_capacity
        rest_fog_1 = fog_capacity[0] - fog_used_capacity[0]
        rest_fog_2 = fog_capacity[1] - fog_used_capacity[1]

        can_allocate = min(
            embb_num,
            rest_cloud // band_division[I][CLOUD],
            max(rest_fog_1 // band_division[I][FOG], rest_fog_2 // band_division[I][FOG]),
        )

        index = -1
        if fog_used_capacity[0] + can_allocate * band_division[I][FOG] <= fog_capacity[0]:
            index = 0
        elif fog_used_capacity[1] + can_allocate * band_division[I][FOG] <= fog_capacity[1]:
            index = 1

        cloud_used_capacity += can_allocate * band_division[I][CLOUD]
        fog_used_capacity[index] += can_allocate * band_division[I][FOG]
        embb_num -= can_allocate

    if embb_num != 0:
        # Allocate as much as possible using split B
        rest_cloud = cloud_capacity - cloud_used_capacity
        rest_fog_1 = fog_capacity[0] - fog_used_capacity[0]
        rest_fog_2 = fog_capacity[1] - fog_used_capacity[1]

        can_allocate = min(
            embb_num,
            rest_cloud // band_division[B][CLOUD],
            max(rest_fog_1 // band_division[B][FOG], rest_fog_2 // band_division[B][FOG]),
        )

        index = -1
        if fog_used_capacity[0] + can_allocate * band_division[B][FOG] <= fog_capacity[0]:
            index = 0
        elif fog_used_capacity[1] + can_allocate * band_division[B][FOG] <= fog_capacity[1]:
            index = 1

        cloud_used_capacity += can_allocate * band_division[B][CLOUD]
        fog_used_capacity[index] += can_allocate * band_division[B][FOG]
        embb_num -= can_allocate

    lost_packeges += embb_num

    # Process mMTC services
    # Allocate as much as possible using split E
    rest_cloud = cloud_capacity - cloud_used_capacity
    can_allocate = min(rest_cloud // band_division[E][CLOUD], mmtc_num)
    cloud_used_capacity += can_allocate * band_division[E][CLOUD]
    mmtc_num -= can_allocate

    if mmtc_num != 0:
        # Allocate as much as possible using split I
        rest_cloud = cloud_capacity - cloud_used_capacity
        rest_fog_1 = fog_capacity[0] - fog_used_capacity[0]
        rest_fog_2 = fog_capacity[1] - fog_used_capacity[1]

        can_allocate = min(
            mmtc_num,
            rest_cloud // band_division[I][CLOUD],
            max(rest_fog_1 // band_division[I][FOG], rest_fog_2 // band_division[I][FOG]),
        )

        index = -1
        if fog_used_capacity[0] + can_allocate * band_division[I][FOG] <= fog_capacity[0]:
            index = 0
        elif fog_used_capacity[1] + can_allocate * band_division[I][FOG] <= fog_capacity[1]:
            index = 1

        cloud_used_capacity += can_allocate * band_division[I][CLOUD]
        fog_used_capacity[index] += can_allocate * band_division[I][FOG]
        mmtc_num -= can_allocate

    if mmtc_num != 0:
        # Allocate as much as possible using split B
        rest_cloud = cloud_capacity - cloud_used_capacity
        rest_fog_1 = fog_capacity[0] - fog_used_capacity[0]
        rest_fog_2 = fog_capacity[1] - fog_used_capacity[1]

        can_allocate = min(
            mmtc_num,
            rest_cloud // band_division[B][CLOUD],
            max(rest_fog_1 // band_division[B][FOG], rest_fog_2 // band_division[B][FOG]),
        )

        index = -1
        if fog_used_capacity[0] + can_allocate * band_division[B][FOG] <= fog_capacity[0]:
            index = 0
        elif fog_used_capacity[1] + can_allocate * band_division[B][FOG] <= fog_capacity[1]:
            index = 1

        cloud_used_capacity += can_allocate * band_division[B][CLOUD]
        fog_used_capacity[index] += can_allocate * band_division[B][FOG]
        mmtc_num -= can_allocate

    lost_packeges += mmtc_num

    # Process URLLC services
    # Cannot use split E
    rest_cloud = cloud_capacity - cloud_used_capacity
    can_allocate = min(rest_cloud // band_division[E][CLOUD], urllc_num)

    if can_allocate == 0:
        # Cannot use split I
        rest_cloud = cloud_capacity - cloud_used_capacity
        rest_fog_1 = fog_capacity[0] - fog_used_capacity[0]
        rest_fog_2 = fog_capacity[1] - fog_used_capacity[1]

        can_allocate = min(
            urllc_num,
            rest_cloud // band_division[I][CLOUD],
            max(rest_fog_1 // band_division[I][FOG], rest_fog_2 // band_division[I][FOG]),
        )

        if can_allocate == 0:
            # Allocate as much as possible using split B
            rest_cloud = cloud_capacity - cloud_used_capacity
            rest_fog_1 = fog_capacity[0] - fog_used_capacity[0]
            rest_fog_2 = fog_capacity[1] - fog_used_capacity[1]

            can_allocate = min(
                urllc_num,
                rest_cloud // band_division[B][CLOUD],
                max(rest_fog_1 // band_division[B][FOG], rest_fog_2 // band_division[B][FOG]),
            )

            index = -1
            if fog_used_capacity[0] + can_allocate * band_division[B][FOG] <= fog_capacity[0]:
                index = 0
            elif fog_used_capacity[1] + can_allocate * band_division[B][FOG] <= fog_capacity[1]:
                index = 1

            cloud_used_capacity += can_allocate * band_division[B][CLOUD]
            fog_used_capacity[index] += can_allocate * band_division[B][FOG]
            urllc_num -= can_allocate

    lost_packeges += urllc_num

    if cloud_used_capacity > 0 and not active["cloud"]:
        total_energy_consumption += cloud_energy
        active["cloud"] = True
    if fog_used_capacity[0] > 0 and not active["fog_1"]:
        total_energy_consumption += fog_energy
        active["fog_1"] = True
    if fog_used_capacity[1] > 0 and not active["fog_2"]:
        total_energy_consumption += fog_energy
        active["fog_2"] = True

    energy_consumption.append(total_energy_consumption)
    cloud_band.append(cloud_used_capacity)
    fog_band.append(fog_used_capacity.copy())
    lost_packeges_all.append(lost_packeges)

In [4]:
print("Cloud band:", cloud_band)
print("Fog band:", fog_band)
print("Lost packages:", lost_packeges_all)
print("Total energy:", energy_consumption)

Cloud band: [9830, 17694, 23592, 25558, 29490, 29490, 37354, 39994.4]
Fog band: [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [1291.6, 0]]
Lost packages: [0, 1, 3, 7, 10, 15, 16, 19.0]
Total energy: [600, 600, 600, 600, 600, 600, 600, 900]
